<a href="https://colab.research.google.com/github/jackevansadl/CHEM3630/blob/main/optfreq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q condacolab

In [ ]:
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:21
🔁 Restarting kernel...


In [ ]:
! mamba install -c anaconda intel-openmp --quiet >/dev/null 2>&1
! mamba install -c psi4 psi4 --quiet >/dev/null 2>&1
! pip install -q ase >/dev/null 2>&1
! pip install -q nglview >/dev/null 2>&1
! pip install -q 'ipywidgets>=7.6.0,<8' --force-reinstall >/dev/null 2>&1

In [ ]:
from ase.build import molecule
from ase import Atoms

water = molecule('H2O')

d=0.97
ohminus = Atoms('OH', positions=([0-d/2,0,0],[0+d/2,0,0]), charges=[-1,0])

hplus = molecule('H')
hplus.set_initial_charges([+1])

molecules = [water, ohminus]

In [ ]:
from ase.calculators.psi4 import Psi4
calc = Psi4(atoms = hplus,
            method = 'b3lyp',
            memory = '500MB', # this is the default, be aware!
            basis = '6-31G**',
            charge = +1,
            multiplicity=1,
            label = hplus.get_chemical_formula())

hplus.calc = calc
print(hplus.get_potential_energy())

/usr/local/lib/python3.8/site-packages/ase/calculators/psi4.py:116: UserWarning: A charge was provided without a spin multiplicity. A multiplicity of 1 is assumed
  warnings.warn('A charge was provided without a spin '


0.0


In [ ]:
from ase.calculators.psi4 import Psi4
from ase.build import molecule
import numpy as np
from ase.optimize import QuasiNewton

optimised_molecules = []
for atoms in molecules:
  print(atoms.get_chemical_formula())
  calc = Psi4(atoms = atoms,
        method = 'b3lyp',
        memory = '500MB', # this is the default, be aware!
        basis = '6-31G**',

        charge = np.sum(atoms.get_initial_charges()),
        multiplicity=1,
        label = atoms.get_chemical_formula())

  atoms.calc = calc

  QuasiNewton(atoms).run(fmax=0.0005)
  print(atoms.get_potential_energy())
  optimised_molecules.append(atoms)

H2O
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 05:57:51    -2079.486357        0.1745
BFGSLineSearch:    1[  2] 05:58:00    -2079.486878        0.0357
BFGSLineSearch:    2[  4] 05:58:11    -2079.486985        0.0021
BFGSLineSearch:    3[  5] 05:58:15    -2079.486985        0.0002
-2079.486985491489
HO
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 05:58:18    -2060.613368        0.6006
BFGSLineSearch:    1[  2] 05:58:27    -2060.617318        0.0187
BFGSLineSearch:    2[  3] 05:58:31    -2060.617322        0.0009
BFGSLineSearch:    3[  4] 05:58:34    -2060.617322        0.0000
-2060.617322387948


In [ ]:
from ase.vibrations import Vibrations
for atoms in optimised_molecules:
  vib = Vibrations(atoms, name=atoms.get_chemical_formula(), delta=0.01)
  vib.clean()
  vib.run()
  vib.write_mode(0)
  vib.summary()

---------------------
  #    meV     cm^-1
---------------------
  0    6.5i     52.6i
  1    0.4i      3.1i
  2    0.2i      1.3i
  3    0.1       0.5
  4    3.4      27.3
  5    3.4      27.8
  6  206.3    1664.2
  7  471.1    3799.4
  8  485.1    3912.6
---------------------
Zero-point energy: 0.585 eV
---------------------
  #    meV     cm^-1
---------------------
  0    0.0i      0.0i
  1    0.0i      0.0i
  2    0.0i      0.0i
  3    3.1      25.2
  4    3.1      25.2
  5  433.8    3499.1
---------------------
Zero-point energy: 0.220 eV


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

from ase.visualize import view
from ase.io import read
atoms = read('/content/H2O.0.traj', index=":")
view(atoms, viewer='ngl')